# Part 1

## Uploading Libararies

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib # library to handle requests
import requests

# import BeautifulSoup
from bs4 import BeautifulSoup

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import lxml
import csv

print('Libraries imported.')

Libraries imported.


## Scraping data and saving in csv file

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#get response from url
res = requests.get(url).text

In [5]:
#scrape
soup = BeautifulSoup(res, 'lxml')
table = soup.find('table')

## Data manupilation

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
#csv
csv_file = open('Toronto.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postal Code','Borough','Neighborhood'])

rowdata = ''
xx = ''
for record in table.find_all('tr'):
    rowdata = ''
    for data in record.find_all('td'):
        rowdata = rowdata + ',' + data.text
    xx = rowdata[1:-1]
    xx = xx.split(',')

    if len(xx) > 1:
       postcode = xx[0]
       borough = xx[1] 
       neighborhood = xx[2]+', '
       if neighborhood == 'Not assigned, ':
          neighborhood = borough + ', '
       if borough != 'Not assigned':
          csv_writer.writerow([postcode,borough,neighborhood])
csv_file.close()

In [7]:
# data procesing

df = pd.read_csv('Toronto.csv')
df2 = df[['Postal Code','Borough']]
df3 = df[['Postal Code','Neighborhood']]
df2 = df2.drop_duplicates(subset=None, keep='first', inplace=False)
df3 = df3.groupby(['Postal Code'], as_index = False).sum()
for i in range(df3.shape[0]):
    xx = df3['Neighborhood'][i]
    xx = xx[0:-2]
    df3['Neighborhood'][i] = xx
df = pd.merge(df2, df3, on='Postal Code')

print('The dataframe has {} boroughs and {} postcodes. \n'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

df.head(25)

The dataframe has 11 boroughs and 103 postcodes. 



,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
print('The dataframe has number of rows equal to:',
        df.shape[0])

The dataframe has number of rows equal to: 103
